In [ ]:
#%matplotlib inline  
import powerlaw
import csv
import glob
from collections import defaultdict
import gc
import sys
from multiprocessing import Pool
#import numpy as np
#import scipy.stats

PUBYEAR_MIN = 1996
PUBYEAR_MAX = 2017

CITEYEAR_MAX = 2017

eid_info_dir = "../DATA/output/docinfo/eid_source_pubdate/"
eid_info_files = glob.glob(eid_info_dir + "*.txt")

pubyear_dict = {}
srcid_dict = {}
srcid_asjc = {}

_MAX_PROCESS = 8


def flatList(tdlist):
    templist = []
    for i in tdlist:
        templist += i
    return templist


In [ ]:
for nowfile in eid_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[1].isdigit()):
                pubyear_dict[row[0]] = int(row[1])
                srcid_dict[row[0]] = row[2]
gc.collect()

print len(pubyear_dict), len(srcid_dict)


In [ ]:
source_info_file = "../DATA/SCOPUS_SCIMAGO_MAPPING_DATA/src_final.txt"

with open(source_info_file, 'rb') as csvfile: 
    reader = csv.reader(csvfile, delimiter='\t') 
    for row in reader:
        srcid_asjc[row[0]] = row[5].split(";")
    #print srcid_asjc.items()
gc.collect()


In [ ]:
PArrayLength = PUBYEAR_MAX - PUBYEAR_MIN + 1
CArrayLength = CITEYEAR_MAX - PUBYEAR_MIN + 1

network_info_dir = "../DATA/output/network/separated/"
network_info_files = glob.glob(network_info_dir + "*.txt")
print len(network_info_files)

CountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    CountArray.append(temparray)

SrcCountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    SrcCountArray.append(temparray)

SrcPaperArray = []
for i in range(PArrayLength):
    SrcPaperArray.append({})    
    
ASJCCountArray = []
for i in range(PArrayLength):
    temparray = []
    for j in range(CArrayLength):
        temparray.append({})
    ASJCCountArray.append(temparray)

gc.collect()
count_exist = 0
#print SrcPaperArray
for k, v in pubyear_dict.iteritems():
    if(k not in srcid_dict):
        continue
    elif(srcid_dict[k] not in srcid_asjc):
        continue
    elif(v > PUBYEAR_MAX or v < PUBYEAR_MIN):
        continue
    SrcPaperArray[v - PUBYEAR_MIN][srcid_dict[k]] = (SrcPaperArray[v - PUBYEAR_MIN]).setdefault(srcid_dict[k], 0) + 1        
    for nC in range(v - PUBYEAR_MIN, CArrayLength):
        count_exist += 1
        for asjcid in srcid_asjc[ srcid_dict[k] ]:
            ASJCCountArray[v - PUBYEAR_MIN][nC][asjcid] = 0
        CountArray[v - PUBYEAR_MIN][nC][k] = 0
        SrcCountArray[v - PUBYEAR_MIN][nC][srcid_dict[k]] = 0       

print "STEP2"
print count_exist
count_err = 0

for nowfile in network_info_files:
    with open(nowfile, 'rb') as csvfile: 
        reader = csv.reader(csvfile, delimiter='\t') 
        for row in reader: 
            if(row[0] not in srcid_dict or row[1] not in srcid_dict):
                pass
            elif(srcid_dict[row[0]] not in srcid_asjc or srcid_dict[row[1]] not in srcid_asjc):
                pass
            elif(pubyear_dict[row[0]] > PUBYEAR_MAX or pubyear_dict[row[0]] < PUBYEAR_MIN):
                pass
            elif(pubyear_dict[row[1]] > CITEYEAR_MAX):
                pass
            elif(pubyear_dict[row[1]] < pubyear_dict[row[0]]):
                count_err += 1 
            else:
                nP = pubyear_dict[row[0]] - PUBYEAR_MIN
                nC = pubyear_dict[row[1]] - PUBYEAR_MIN
                CountArray[nP][nC][row[0]] += 1
                for asjcid in srcid_asjc[ srcid_dict[row[0]] ]:
                    SrcCountArray[nP][nC][srcid_dict[row[0]]] += 1
                    ASJCCountArray[nP][nC][asjcid] += 1
                SrcCountArray[nP][nC][srcid_dict[row[0]]] += 1
                
gc.collect()
print len(CountArray[1][1].keys())
print count_err

In [ ]:
outfile = "../DATA/pandas_outdegree_out/yearly_count_normalized.tsv"
f_out = open(outfile, "w")
f_out.write("\t".join(["Index", "PubYear", "CiteYear", "DocID", "ASJCCODE", "CiteCount"])+"\n")

In [ ]:
index = 0
for i in range(PArrayLength):
    for j in range(CArrayLength):
        print i, j
        temparray = []
        for key, val in CountArray[i][j].iteritems():
            try:
                val = float(val*SrcPaperArray[i][srcid_dict[k]])/float(SrcCountArray[i][j][srcid_dict[key]])
            except ZeroDivisionError: val = 0
            index += 1
            f_out.write("\t".join([str(index), str(i), str(j), str(key), "NON", str(val)])+"\n")
            for asjcid in srcid_asjc[srcid_dict[key]]:
                f_out.write("\t".join([str(index), str(i), str(j), str(key), str(asjcid), str(val)])+"\n")